<a href="https://colab.research.google.com/github/ThomasL642/Thomas/blob/main/Celeb_A_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import keras
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm
import tensorflow as tf
import glob

from keras.optimizers import Adam
from keras.layers import Input
from keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Reshape, Flatten, Cropping2D, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras import initializers
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2DTranspose, Conv2D

np.random.seed(10)
random_dim = 100
DATADIR = "/content/drive/MyDrive/Colab Notebooks/DATA/Celeb data/img_align_celeba"
IMG_SIZE = 50

In [ ]:
#training_data = []

#images = glob.glob('/content/drive/MyDrive/Colab Notebooks/DATA/Celeb data/img_align_celeba/*.jpg')
#def create_training_data():
#  for img in tqdm(os.listdir(DATADIR)):
#    image = cv2.imread(img)
#    img_array = cv2.imread(os.path.join(DATADIR,img))
#    new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE), interpolation = cv2.INTER_NEAREST)
#    training_data.append(new_array)
#    plt.imshow(training_data[0])
#    plt.show

#create_training_data()

In [ ]:
x_train = np.load('/content/drive/MyDrive/Colab Notebooks/DATA/CelebANumpy.npy')

In [ ]:
#print(len(training_data))

In [ ]:
def load_data():
  x_train = x_train
  x_train = (x_train.astype(np.float32) - 127.5)/127.5
  return x_train

In [ ]:
def get_optimizer():
  return Adam(lr=0.00001, beta_1=0.9)

def get_generator(optimizer):
  alpha = 0.2

  generator = Sequential()
  generator.add(Dense(2*2*512, input_dim = random_dim,kernel_initializer=initializers.RandomNormal(stddev=0.02)))

  generator.add(Reshape((-1, 2, 2, 512)))
  generator.add(BatchNormalization)
  generator.add(Maximum(alpha = 0.2))

  generator.add(Conv2DTranspose(256, 5, 2, padding="VALID"))
  generator.add(BatchNormalization())
  generator.add(Maximum(alpha = 0.2))
  
  generator.add(Conv2DTranspose(128, 5, 2, padding="SAME"))
  generator.add(BatchNormalization())
  generator.add(Maximum(alpha = 0.2))

  generator.add(Conv2DTranspose(out_channel_dim, 5, 2, padding="SAME", activatins = "tanh"))
  generator.compile(loss="BinaryCrossentropy", optimizer=optimizer)

  return generator

def get_discriminator(optimizer):

  alpha = 0.2
  discriminator = Sequential()

  discriminator.add(Conv2D(images, 64, 5, 2, "SAME"))
  discriminator.add(Maximum(alpha = 0.2))

  discriminator.add(Conv2D(128, 5, 2, "SAME"))
  discriminator.add(BatchNormalization())
  discriminator.add(Maximum(alpha = 0.2))

  discriminator.add(Conv2D(lrelu2, 256, 5, 1, "SAME"))
  discriminator.add(BatchNormalization())
  discriminator.add(Maximum(alpha = 0.2)) 

  discriminator.add(Reshape((-1, 4*4*256)))

  discriminator.add(Dense(1, activation = "sigmoid"))
  discriminator.compile(loss="binary_crossentropy", optimizer=optimizer)

  return discriminator

In [ ]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
  discriminator.trainable = False
  gan_input = Input(shape=(random_dim,))
  x = generator(gan_input)
  gan_output = discriminator(x)
  gan = Model(inputs=gan_input, outputs=gan_output)
  gan.compile(loss="binary_crossentropy", optimizer=optimizer)
  return gan

In [ ]:
def plot_generated_images(epoch, generator, examples=100, dim=(10, 10), figsize=(10, 10)):
  noise = np.random.normal(0, 1, size=[examples, random_dim])
  generated_images = generator.predict(noise)
  generated_images = generated_images.reshape(examples, , 28)

  plt.figure(figsize=figsize)
  for i in range(generated_images.shape[0]):
    plt.subplot(dim[0], dim[1], i+1)
    plt.imshow(generated_images[i], interpolation="nearest")
    plt.axis("off")
  plt.tight_layout()
  plt.savefig("gan_generated_image_epoch_%d.png" % epoch)

In [ ]:
def train(epochs=1, batch_size=50):
  batch_count = x_train.shape[0] / batch_size
  adam = get_optimizer()
  generator = get_generator(adam)
  discriminator = get_discriminator(adam)
  gan = get_gan_network(discriminator, random_dim, generator, adam)
  #random noise and images
  for e in range(1, epochs+1):
    print("-"*15,"Epoch %d" % e, "-"*15)
    for _ in tqdm(range(int(batch_count))):
      noise = np.random.normal(0, 1, size=[batch_size, random_dim])
      image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
      
      #generate fake images
      generated_images = generator.predict(noise)
      X = np.concatenate([image_batch, generated_images])

      #labels for fake + real
      y_dis = np.zeros(2*batch_size)
      y_dis[:batch_size] = 0.9

      discriminator.trainable = True 
      discriminator.train_on_batch(X, y_dis)

      noise = np.random.normal(0, 1, size=[batch_size, random_dim])
      y_gen = np.ones(batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise, y_gen)

    if e == 1 or e % 20 == 0:
      plot_generated_images(e, generator)

In [ ]:
train(600, 50)

TypeError: ignored